In [2]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle

### d=600, n=500

In [4]:
## Homoscedastic case
d = 600
n = 500
with pd.ExcelWriter('./Results/Cirsym_Full_Results1.xlsx') as writer:
    for i in range(4):
        if i == 0:
            ## x0
            x = np.zeros((d,))
            x[0] = 1
        if i == 1:
            ## x1
            x = np.zeros((d,))
            x[0] = 1
            x[1] = 1/2
            x[2] = 1/4
            x[6] = 1/2
            x[7] = 1/8
        if i == 2:
            ## x2
            x = np.zeros((d,))
            x[99] = 1
        if i == 3:
            ## x3
            x = 1/np.linspace(1, d, d)**2
        for k in range(3):
            if k == 0:
                s_beta = 5
                beta_0 = np.zeros((d,))
                beta_0[:s_beta] = 1
            if k == 1:
                beta_0 = 1/np.sqrt(np.linspace(1, d, d))
                beta_0 = 5*beta_0/np.linalg.norm(beta_0)
            if k == 2:
                beta_0 = 1/np.linspace(1, d, d)
                beta_0 = 5*beta_0/np.linalg.norm(beta_0)
    
            # True regression function
            m_true = np.dot(x, beta_0)
            
            debl_res1 = pd.read_csv('./Results/debl_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
            lproj_res1 = pd.read_csv('./Results/lproj_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
            refit_res1 = pd.read_csv('./Results/refit_Cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
            deb_est_1se = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_1se_x'+str(i)+'_beta'+str(k)+'.csv')
            if (i == 0) or (i == 2):
                rproj_res1 = pd.read_csv('./Results/rproj_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
                
            # Bias Comparisons
            ## Debiased lasso (Javanmard and Montarani, 2014)
            debl_obs1 = np.mean(abs(debl_res1['m_obs1'] - m_true))
            debl_obs_std1 = np.std(abs(debl_res1['m_obs1'] - m_true))
            # Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', debl_obs1, debl_obs_std1]
            Bias1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                  'Avg bias': [debl_obs1], 'Std': [debl_obs_std1]})
            
            debl_ipw1 = np.mean(abs(debl_res1['m_ipw1'] - m_true))
            debl_ipw_std1 = np.std(abs(debl_res1['m_ipw1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', debl_ipw1, debl_ipw_std1]
            
            debl_obs2 = np.mean(abs(debl_res1['m_obs2'] - m_true))
            debl_obs_std2 = np.std(abs(debl_res1['m_obs2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', debl_obs2, debl_obs_std2]
            
            debl_ipw2 = np.mean(abs(debl_res1['m_ipw2'] - m_true))
            debl_ipw_std2 = np.std(abs(debl_res1['m_ipw2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', debl_ipw2, debl_ipw_std2]
            
            debl_full = np.mean(abs(debl_res1['m_full'] - m_true))
            debl_full_std = np.std(abs(debl_res1['m_full'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', debl_full, debl_full_std]
            
            ## Debiased lasso (van de geer et al., 2014)
            lproj_obs1 = np.mean(abs(lproj_res1['m_obs1'] - m_true))
            lproj_obs_std1 = np.std(abs(lproj_res1['m_obs1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (van de geer) (MCAR, Observed)', lproj_obs1, lproj_obs_std1]

            lproj_ipw1 = np.mean(abs(lproj_res1['m_ipw1'] - m_true))
            lproj_ipw_std1 = np.std(abs(lproj_res1['m_ipw1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (van de geer) (MCAR, IPW)', lproj_ipw1, lproj_ipw_std1]

            lproj_obs2 = np.mean(abs(lproj_res1['m_obs2'] - m_true))
            lproj_obs_std2 = np.std(abs(lproj_res1['m_obs2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (van de geer) (MAR, Observed)', lproj_obs2, lproj_obs_std2]

            lproj_ipw2 = np.mean(abs(lproj_res1['m_ipw2'] - m_true))
            lproj_ipw_std2 = np.std(abs(lproj_res1['m_ipw2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (van de geer) (MAR, IPW)', lproj_ipw2, lproj_ipw_std2]

            lproj_full = np.mean(abs(lproj_res1['m_full'] - m_true))
            lproj_full_std = np.std(abs(lproj_res1['m_full'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (van de geer) (Full oracle data)', lproj_full, lproj_full_std]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                rproj_obs1 = np.mean(abs(rproj_res1['m_obs1'] - m_true))
                rproj_obs_std1 = np.std(abs(rproj_res1['m_obs1'] - m_true))
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, Observed)', rproj_obs1, rproj_obs_std1]
                
                rproj_ipw1 = np.mean(abs(rproj_res1['m_ipw1'] - m_true))
                rproj_ipw_std1 = np.std(abs(rproj_res1['m_ipw1'] - m_true))
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, IPW)', rproj_ipw1, rproj_ipw_std1]
                
                rproj_obs2 = np.mean(abs(rproj_res1['m_obs2'] - m_true))
                rproj_obs_std2 = np.std(abs(rproj_res1['m_obs2'] - m_true))
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, Observed)', rproj_obs2, rproj_obs_std2]
                
                rproj_ipw2 = np.mean(abs(rproj_res1['m_ipw2'] - m_true))
                rproj_ipw_std2 = np.std(abs(rproj_res1['m_ipw2'] - m_true))
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, IPW)', rproj_ipw2, rproj_ipw_std2]
                
                rproj_full = np.mean(abs(rproj_res1['m_full'] - m_true))
                rproj_full_std = np.std(abs(rproj_res1['m_full'] - m_true))
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (Full oracle data)', rproj_full, rproj_full_std]
            
            ## Lasso Refitting
            refit_obs1 = np.mean(abs(refit_res1['m_obs1'] - m_true))
            refit_obs_std1 = np.std(abs(refit_res1['m_obs1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, Observed)', refit_obs1, refit_obs_std1]
            
            refit_ipw1 = np.mean(abs(refit_res1['m_ipw1'] - m_true))
            refit_ipw_std1 = np.std(abs(refit_res1['m_ipw1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, IPW)', refit_ipw1, refit_ipw_std1]
            
            refit_obs2 = np.mean(abs(refit_res1['m_obs2'] - m_true))
            refit_obs_std2 = np.std(abs(refit_res1['m_obs2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, Observed)', refit_obs2, refit_obs_std2]
            
            refit_ipw2 = np.mean(abs(refit_res1['m_ipw2'] - m_true))
            refit_ipw_std2 = np.std(abs(refit_res1['m_ipw2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, IPW)', refit_ipw2, refit_ipw_std2]
            
            refit_full = np.mean(abs(refit_res1['m_full'] - m_true))
            refit_full_std = np.std(abs(refit_res1['m_full'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (Full oracle data)', refit_full, refit_full_std]
            
            ## Our proposed debiasing framework
            m1 = np.mean(abs(deb_est_1se['m_deb1'] - m_true))
            m_std1 = np.std(abs(deb_est_1se['m_deb1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, 1SE)', m1, m_std1]
            
            m2 = np.mean(abs(deb_est_1se['m_deb2'] - m_true))
            m_std2 = np.std(abs(deb_est_1se['m_deb2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, 1SE)', m2, m_std2]
            
            
            # Coverage Probability
            ## Debiased lasso (Javanmard and Montarani, 2014)
            cov_prob = (debl_res1['m_obs1'] + debl_res1['ci_len_obs1']/2 >= m_true) & \
            (debl_res1['m_obs1'] - debl_res1['ci_len_obs1']/2 <= m_true)
            # Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            Cov_prob1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                      'Coverage Probability': [np.mean(cov_prob)], 
                                      'Std': [np.std(cov_prob)]})
            
            cov_prob = (debl_res1['m_ipw1'] + debl_res1['ci_len_ipw1']/2 >= m_true) & \
            (debl_res1['m_ipw1'] - debl_res1['ci_len_ipw1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (debl_res1['m_obs2'] + debl_res1['ci_len_obs2']/2 >= m_true) & \
            (debl_res1['m_obs2'] - debl_res1['ci_len_obs2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (debl_res1['m_ipw2'] + debl_res1['ci_len_ipw2']/2 >= m_true) & \
            (debl_res1['m_ipw2'] - debl_res1['ci_len_ipw2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (debl_res1['m_full'] + debl_res1['ci_len_full']/2 >= m_true) & \
            (debl_res1['m_full'] - debl_res1['ci_len_full']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (Full data)', np.mean(cov_prob), np.std(cov_prob)]
            
            
            ## Debiased lasso (van de geer et al., 2014)
            cov_prob = (lproj_res1['m_obs1'] + np.sqrt(lproj_res1['asym_se_obs1'])*lproj_res1['sigma_hat_obs1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (lproj_res1['m_obs1'] - np.sqrt(lproj_res1['asym_se_obs1'])*lproj_res1['sigma_hat_obs1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (van de geer) (MCAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]

            cov_prob = (lproj_res1['m_ipw1'] + np.sqrt(lproj_res1['asym_se_ipw1'])*lproj_res1['sigma_hat_ipw1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (lproj_res1['m_ipw1'] - np.sqrt(lproj_res1['asym_se_ipw1'])*lproj_res1['sigma_hat_ipw1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (van de geer) (MCAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]

            cov_prob = (lproj_res1['m_obs2'] + np.sqrt(lproj_res1['asym_se_obs2'])*lproj_res1['sigma_hat_obs2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (lproj_res1['m_obs2'] - np.sqrt(lproj_res1['asym_se_obs2'])*lproj_res1['sigma_hat_obs2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (van de geer) (MAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]

            cov_prob = (lproj_res1['m_ipw2'] + np.sqrt(lproj_res1['asym_se_ipw2'])*lproj_res1['sigma_hat_ipw2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (lproj_res1['m_ipw2'] - np.sqrt(lproj_res1['asym_se_ipw2'])*lproj_res1['sigma_hat_ipw2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (van de geer) (MAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]

            cov_prob = (lproj_res1['m_full'] + np.sqrt(lproj_res1['asym_se_full'])*lproj_res1['sigma_hat_full']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (lproj_res1['m_full'] - np.sqrt(lproj_res1['asym_se_full'])*lproj_res1['sigma_hat_full']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (van de geer) (MAR, Full data)', np.mean(cov_prob), np.std(cov_prob)]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                cov_prob = (rproj_res1['m_obs1'] + rproj_res1['ci_len_obs1']/2 >= m_true) & \
                (rproj_res1['m_obs1'] - rproj_res1['ci_len_obs1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob)]
                
                cov_prob = (rproj_res1['m_ipw1'] + rproj_res1['ci_len_ipw1']/2 >= m_true) & \
                (rproj_res1['m_ipw1'] - rproj_res1['ci_len_ipw1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob)]
                
                cov_prob = (rproj_res1['m_obs2'] + rproj_res1['ci_len_obs2']/2 >= m_true) & \
                (rproj_res1['m_obs2'] - rproj_res1['ci_len_obs2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob)]
                
                cov_prob = (rproj_res1['m_ipw2'] + rproj_res1['ci_len_ipw2']/2 >= m_true) & \
                (rproj_res1['m_ipw2'] - rproj_res1['ci_len_ipw2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob)]
                
                cov_prob = (rproj_res1['m_full'] + rproj_res1['ci_len_full']/2 >= m_true) & \
                (rproj_res1['m_full'] - rproj_res1['ci_len_full']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (Full data)', np.mean(cov_prob), 
                                                       np.std(cov_prob)]
            
            ## Lasso refitting
            cov_prob = (refit_res1['m_obs1'] + refit_res1['ci_len_obs1']/2 >= m_true) & \
            (refit_res1['m_obs1'] - refit_res1['ci_len_obs1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_ipw1'] + refit_res1['ci_len_ipw1']/2 >= m_true) & \
            (refit_res1['m_ipw1'] - refit_res1['ci_len_ipw1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_obs2'] + refit_res1['ci_len_obs2']/2 >= m_true) & \
            (refit_res1['m_obs2'] - refit_res1['ci_len_obs2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_ipw2'] + refit_res1['ci_len_ipw2']/2 >= m_true) & \
            (refit_res1['m_ipw2'] - refit_res1['ci_len_ipw2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_full'] + refit_res1['ci_len_full']/2 >= m_true) & \
            (refit_res1['m_full'] - refit_res1['ci_len_full']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (Full data)', np.mean(cov_prob), np.std(cov_prob)]
            
            # Proposed framework        
            cov_prob = (deb_est_1se['m_deb1'] + deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_1se['m_deb1'] - deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, 1SE)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (deb_est_1se['m_deb2'] + deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_1se['m_deb2'] - deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, 1SE)', np.mean(cov_prob), np.std(cov_prob)]
            
            
            # Lengths of confidence intervals
            ## Debiased lasso (Javanmard and Montarani, 2014)
            CI_len1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                            'CI length': [np.mean(debl_res1['ci_len_obs1'])], 
                            'Std': [np.std(debl_res1['ci_len_obs1'])]})
    #         CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(debl_res1['ci_len_obs1']), 
    #                                            np.std(debl_res1['ci_len_obs1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(debl_res1['ci_len_ipw1']), 
                                               np.std(debl_res1['ci_len_ipw1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(debl_res1['ci_len_obs2']), 
                                               np.std(debl_res1['ci_len_obs2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(debl_res1['ci_len_ipw2']), 
                                               np.std(debl_res1['ci_len_ipw2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', np.mean(debl_res1['ci_len_full']), 
                                               np.std(debl_res1['ci_len_full'])]
            
            ## Debiased lasso (van de geer et al., 2014)
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (van de geer) (MCAR, Observed)', 
                                               np.mean(2*np.sqrt(lproj_res1['asym_se_obs1'])*lproj_res1['sigma_hat_obs1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*np.sqrt(lproj_res1['asym_se_obs1'])*lproj_res1['sigma_hat_obs1']*scipy.stats.norm.ppf(1-0.05/2))]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (van de geer) (MCAR, IPW)', 
                                               np.mean(2*np.sqrt(lproj_res1['asym_se_ipw1'])*lproj_res1['sigma_hat_ipw1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*np.sqrt(lproj_res1['asym_se_ipw1'])*lproj_res1['sigma_hat_ipw1']*scipy.stats.norm.ppf(1-0.05/2))]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (van de geer) (MAR, Observed)', 
                                               np.mean(2*np.sqrt(lproj_res1['asym_se_obs2'])*lproj_res1['sigma_hat_obs2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*np.sqrt(lproj_res1['asym_se_obs2'])*lproj_res1['sigma_hat_obs2']*scipy.stats.norm.ppf(1-0.05/2))]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (van de geer) (MAR, IPW)', 
                                               np.mean(2*np.sqrt(lproj_res1['asym_se_ipw2'])*lproj_res1['sigma_hat_ipw2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*np.sqrt(lproj_res1['asym_se_ipw2'])*lproj_res1['sigma_hat_ipw2']*scipy.stats.norm.ppf(1-0.05/2))]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (van de geer) (Full oracle data)', 
                                               np.mean(2*np.sqrt(lproj_res1['asym_se_full'])*lproj_res1['sigma_hat_full']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*np.sqrt(lproj_res1['asym_se_full'])*lproj_res1['sigma_hat_full']*scipy.stats.norm.ppf(1-0.05/2))]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, Observed)', 
                                                   np.mean(rproj_res1['ci_len_obs1']), 
                                                   np.std(rproj_res1['ci_len_obs1'])]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(rproj_res1['ci_len_ipw1']), 
                                                   np.std(rproj_res1['ci_len_ipw1'])]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, Observed)', np.mean(rproj_res1['ci_len_obs2']), 
                                                   np.std(rproj_res1['ci_len_obs2'])]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, IPW)', np.mean(rproj_res1['ci_len_ipw2']), 
                                                   np.std(rproj_res1['ci_len_ipw2'])]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (Full oracle data)', np.mean(rproj_res1['ci_len_full']), 
                                                   np.std(rproj_res1['ci_len_full'])]
            
            ## Lasso refitting
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(refit_res1['ci_len_obs1']), 
                                               np.std(refit_res1['ci_len_obs1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(refit_res1['ci_len_ipw1']), 
                                               np.std(refit_res1['ci_len_ipw1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(refit_res1['ci_len_obs2']), 
                                               np.std(refit_res1['ci_len_obs2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(refit_res1['ci_len_ipw2']), 
                                               np.std(refit_res1['ci_len_ipw2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (Full oracle data)', np.mean(refit_res1['ci_len_full']), 
                                               np.std(refit_res1['ci_len_full'])]
            
            ## Proposed debiasing framework
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, 1SE)', 
                                               np.mean(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))]
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, 1SE)', 
                                               np.mean(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))]
            
            full_res1 = pd.concat([Bias1, Cov_prob1[['Coverage Probability', 'Std']], CI_len1[['CI length', 'Std']]], axis=1)
            
            # full_res1.to_csv('./Results/Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv', index=False)
            full_res1.to_excel(writer, sheet_name='Cirsym_cov_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k), index=False)

In [17]:
deb_est_mincv.shape

(997, 6)

### d=1000, n=900 (Gaussian noise)

In [3]:
## Homoscedastic case (Gaussian noises)
d = 1000
n = 900
with pd.ExcelWriter('./Results/Cirsym_Full_Results_d'+str(d)+'_n'+str(n)+'_Gaussian_noise.xlsx') as writer:
    for i in range(5):
        if i == 0:
            ## x0
            x = np.zeros((d,))
            x[0] = 1
        if i == 1:
            ## x1
            x = np.zeros((d,))
            x[0] = 1
            x[1] = 1/2
            x[2] = 1/4
            x[6] = 1/2
            x[7] = 1/8
        if i == 2:
            ## x2
            x = np.zeros((d,))
            x[99] = 1
        if i == 3:
            ## x3
            x = 1/np.linspace(1, d, d)
        if i == 4:
            ## x4
            x = 1/np.linspace(1, d, d)**2
        for k in range(3):
            if k == 0:
                s_beta = 5
                beta_0 = np.zeros((d,))
                beta_0[:s_beta] = np.sqrt(5)
            if k == 1:
                beta_0 = 1/np.sqrt(np.linspace(1, d, d))
                beta_0 = 5*beta_0/np.linalg.norm(beta_0)
            if k == 2:
                beta_0 = 1/np.linspace(1, d, d)
                beta_0 = 5*beta_0/np.linalg.norm(beta_0)
    
            # True regression function
            m_true = np.dot(x, beta_0)
            
            debl_res1 = pd.read_csv('./Results/debl_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
            refit_res1 = pd.read_csv('./Results/refit_Cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
            deb_est_1se = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_1se.csv')
            deb_est_mincv = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_mincv.csv')
            deb_est_minfeas = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_minfeas.csv')
            if (i == 0) or (i == 2):
                rproj_res1 = pd.read_csv('./Results/rproj_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv')
                
            # Bias Comparisons
            ## Debiased lasso (Javanmard and Montarani, 2014)
            debl_obs1 = np.mean(abs(debl_res1['m_obs1'] - m_true))
            debl_obs_std1 = np.std(abs(debl_res1['m_obs1'] - m_true))
            debl_obs_ste1 = debl_obs_std1/np.sqrt(1000)
            # Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', debl_obs1, debl_obs_std1, debl_obs_ste1]
            Bias1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                  'Avg bias': [debl_obs1], 'Std': [debl_obs_std1], 'StdErr': [debl_obs_ste1]})
            
            debl_ipw1 = np.mean(abs(debl_res1['m_ipw1'] - m_true))
            debl_ipw_std1 = np.std(abs(debl_res1['m_ipw1'] - m_true))
            debl_ipw_ste1 = debl_ipw_std1/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', debl_ipw1, 
                                           debl_ipw_std1, debl_ipw_ste1]
            
            debl_obs2 = np.mean(abs(debl_res1['m_obs2'] - m_true))
            debl_obs_std2 = np.std(abs(debl_res1['m_obs2'] - m_true))
            debl_obs_ste2 = debl_obs_std2 / np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', debl_obs2, 
                                           debl_obs_std2, debl_obs_ste2]
            
            debl_ipw2 = np.mean(abs(debl_res1['m_ipw2'] - m_true))
            debl_ipw_std2 = np.std(abs(debl_res1['m_ipw2'] - m_true))
            debl_ipw_ste2 = debl_ipw_std2/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', debl_ipw2, 
                                           debl_ipw_std2, debl_ipw_ste2]
            
            debl_full = np.mean(abs(debl_res1['m_full'] - m_true))
            debl_full_std = np.std(abs(debl_res1['m_full'] - m_true))
            debl_full_ste = debl_full_std/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', debl_full, 
                                           debl_full_std, debl_full_ste]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                rproj_obs1 = np.mean(abs(rproj_res1['m_obs1'] - m_true))
                rproj_obs_std1 = np.std(abs(rproj_res1['m_obs1'] - m_true))
                rproj_obs_ste1 = rproj_obs_std1/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, Observed)', rproj_obs1, rproj_obs_std1, 
                                               rproj_obs_ste1]
                
                rproj_ipw1 = np.mean(abs(rproj_res1['m_ipw1'] - m_true))
                rproj_ipw_std1 = np.std(abs(rproj_res1['m_ipw1'] - m_true))
                rproj_ipw_ste1 = rproj_ipw_std1/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, IPW)', rproj_ipw1, rproj_ipw_std1, rproj_ipw_ste1]
                
                rproj_obs2 = np.mean(abs(rproj_res1['m_obs2'] - m_true))
                rproj_obs_std2 = np.std(abs(rproj_res1['m_obs2'] - m_true))
                rproj_obs_ste2 = rproj_obs_std2/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, Observed)', rproj_obs2, rproj_obs_std2, 
                                               rproj_obs_ste2]
                
                rproj_ipw2 = np.mean(abs(rproj_res1['m_ipw2'] - m_true))
                rproj_ipw_std2 = np.std(abs(rproj_res1['m_ipw2'] - m_true))
                rproj_ipw_ste2 = rproj_ipw_std2/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, IPW)', rproj_ipw2, rproj_ipw_std2, rproj_ipw_ste2]
                
                rproj_full = np.mean(abs(rproj_res1['m_full'] - m_true))
                rproj_full_std = np.std(abs(rproj_res1['m_full'] - m_true))
                rproj_full_ste = rproj_full_std / np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Ridge projection (Full oracle data)', rproj_full, 
                                               rproj_full_std, rproj_full_ste]
            
            ## Lasso Refitting
            refit_obs1 = np.mean(abs(refit_res1['m_obs1'] - m_true))
            refit_obs_std1 = np.std(abs(refit_res1['m_obs1'] - m_true))
            refit_obs_ste1 = refit_obs_std1/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, Observed)', refit_obs1, refit_obs_std1, refit_obs_ste1]
            
            refit_ipw1 = np.mean(abs(refit_res1['m_ipw1'] - m_true))
            refit_ipw_std1 = np.std(abs(refit_res1['m_ipw1'] - m_true))
            refit_ipw_ste1 = refit_ipw_std1/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, IPW)', refit_ipw1, refit_ipw_std1, refit_ipw_ste1]
            
            refit_obs2 = np.mean(abs(refit_res1['m_obs2'] - m_true))
            refit_obs_std2 = np.std(abs(refit_res1['m_obs2'] - m_true))
            refit_obs_ste2 = refit_obs_std2/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, Observed)', refit_obs2, refit_obs_std2, refit_obs_ste2]
            
            refit_ipw2 = np.mean(abs(refit_res1['m_ipw2'] - m_true))
            refit_ipw_std2 = np.std(abs(refit_res1['m_ipw2'] - m_true))
            refit_ipw_ste2 = refit_ipw_std2/np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, IPW)', refit_ipw2, refit_ipw_std2, refit_obs_ste2]
            
            refit_full = np.mean(abs(refit_res1['m_full'] - m_true))
            refit_full_std = np.std(abs(refit_res1['m_full'] - m_true))
            refit_full_ste = refit_full_std / np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (Full oracle data)', refit_full, refit_full_std, refit_full_ste]
            
            ## Our proposed debiasing framework
            m1 = np.mean(abs(deb_est_mincv['m_deb1'] - m_true))
            m_std1 = np.std(abs(deb_est_mincv['m_deb1'] - m_true))
            m_ste1 = m_std1 /np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, Mincv)', m1, m_std1, m_ste1]
            
            m2 = np.mean(abs(deb_est_mincv['m_deb2'] - m_true))
            m_std2 = np.std(abs(deb_est_mincv['m_deb2'] - m_true))
            m_ste2 = m_std2 / np.sqrt(1000) 
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, Mincv)', m2, m_std2, m_ste2]
            
            m1 = np.mean(abs(deb_est_1se['m_deb1'] - m_true))
            m_std1 = np.std(abs(deb_est_1se['m_deb1'] - m_true))
            m_ste1 = m_std1 /np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, 1SE)', m1, m_std1, m_ste1]
            
            m2 = np.mean(abs(deb_est_1se['m_deb2'] - m_true))
            m_std2 = np.std(abs(deb_est_1se['m_deb2'] - m_true))
            m_ste2 = m_std2 / np.sqrt(1000) 
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, 1SE)', m2, m_std2, m_ste2]
            
            m1 = np.mean(abs(deb_est_minfeas['m_deb1'] - m_true))
            m_std1 = np.std(abs(deb_est_minfeas['m_deb1'] - m_true))
            m_ste1 = m_std1 /np.sqrt(1000)
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, Min-feas)', m1, m_std1, m_ste1]
            
            m2 = np.mean(abs(deb_est_minfeas['m_deb2'] - m_true))
            m_std2 = np.std(abs(deb_est_minfeas['m_deb2'] - m_true))
            m_ste2 = m_std2 / np.sqrt(1000) 
            Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, Min-feas)', m2, m_std2, m_ste2]
            
            
            # Coverage Probability
            ## Debiased lasso (Javanmard and Montarani, 2014)
            cov_prob = (debl_res1['m_obs1'] + debl_res1['ci_len_obs1']/2 >= m_true) & \
            (debl_res1['m_obs1'] - debl_res1['ci_len_obs1']/2 <= m_true)
            # Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            Cov_prob1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                      'Coverage Probability': [np.mean(cov_prob)], 
                                      'Std': [np.std(cov_prob)], 
                                      'StdErr': [np.std(cov_prob)/np.sqrt(1000)]})
            
            cov_prob = (debl_res1['m_ipw1'] + debl_res1['ci_len_ipw1']/2 >= m_true) & \
            (debl_res1['m_ipw1'] - debl_res1['ci_len_ipw1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (debl_res1['m_obs2'] + debl_res1['ci_len_obs2']/2 >= m_true) & \
            (debl_res1['m_obs2'] - debl_res1['ci_len_obs2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (debl_res1['m_ipw2'] + debl_res1['ci_len_ipw2']/2 >= m_true) & \
            (debl_res1['m_ipw2'] - debl_res1['ci_len_ipw2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (debl_res1['m_full'] + debl_res1['ci_len_full']/2 >= m_true) & \
            (debl_res1['m_full'] - debl_res1['ci_len_full']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (Full data)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                cov_prob = (rproj_res1['m_obs1'] + rproj_res1['ci_len_obs1']/2 >= m_true) & \
                (rproj_res1['m_obs1'] - rproj_res1['ci_len_obs1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (rproj_res1['m_ipw1'] + rproj_res1['ci_len_ipw1']/2 >= m_true) & \
                (rproj_res1['m_ipw1'] - rproj_res1['ci_len_ipw1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (rproj_res1['m_obs2'] + rproj_res1['ci_len_obs2']/2 >= m_true) & \
                (rproj_res1['m_obs2'] - rproj_res1['ci_len_obs2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (rproj_res1['m_ipw2'] + rproj_res1['ci_len_ipw2']/2 >= m_true) & \
                (rproj_res1['m_ipw2'] - rproj_res1['ci_len_ipw2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (rproj_res1['m_full'] + rproj_res1['ci_len_full']/2 >= m_true) & \
                (rproj_res1['m_full'] - rproj_res1['ci_len_full']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (Full data)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            ## Lasso refitting
            cov_prob = (refit_res1['m_obs1'] + refit_res1['ci_len_obs1']/2 >= m_true) & \
            (refit_res1['m_obs1'] - refit_res1['ci_len_obs1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (refit_res1['m_ipw1'] + refit_res1['ci_len_ipw1']/2 >= m_true) & \
            (refit_res1['m_ipw1'] - refit_res1['ci_len_ipw1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (refit_res1['m_obs2'] + refit_res1['ci_len_obs2']/2 >= m_true) & \
            (refit_res1['m_obs2'] - refit_res1['ci_len_obs2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (refit_res1['m_ipw2'] + refit_res1['ci_len_ipw2']/2 >= m_true) & \
            (refit_res1['m_ipw2'] - refit_res1['ci_len_ipw2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (refit_res1['m_full'] + refit_res1['ci_len_full']/2 >= m_true) & \
            (refit_res1['m_full'] - refit_res1['ci_len_full']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (Full data)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            # Proposed framework   
            cov_prob = (deb_est_mincv['m_deb1'] + deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_mincv['m_deb1'] - deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, Mincv)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (deb_est_mincv['m_deb2'] + deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_mincv['m_deb2'] - deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, Mincv)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (deb_est_1se['m_deb1'] + deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_1se['m_deb1'] - deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, 1SE)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (deb_est_1se['m_deb2'] + deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_1se['m_deb2'] - deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, 1SE)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (deb_est_minfeas['m_deb1'] + deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_minfeas['m_deb1'] - deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, Min-feas)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
            cov_prob = (deb_est_minfeas['m_deb2'] + deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
            (deb_est_minfeas['m_deb2'] - deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, Min-feas)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
        
            
            # Lengths of confidence intervals
            ## Debiased lasso (Javanmard and Montarani, 2014)
            CI_len1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                            'CI length': [np.mean(debl_res1['ci_len_obs1'])], 
                            'Std': [np.std(debl_res1['ci_len_obs1'])], 
                            'StdErr': [np.std(debl_res1['ci_len_obs1'])/np.sqrt(1000)]})
    #         CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(debl_res1['ci_len_obs1']), 
    #                                            np.std(debl_res1['ci_len_obs1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(debl_res1['ci_len_ipw1']), 
                                               np.std(debl_res1['ci_len_ipw1']), 
                                               np.std(debl_res1['ci_len_ipw1'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(debl_res1['ci_len_obs2']), 
                                               np.std(debl_res1['ci_len_obs2']), 
                                               np.std(debl_res1['ci_len_obs2'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(debl_res1['ci_len_ipw2']), 
                                               np.std(debl_res1['ci_len_ipw2']), 
                                               np.std(debl_res1['ci_len_ipw2'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', np.mean(debl_res1['ci_len_full']), 
                                               np.std(debl_res1['ci_len_full']), np.std(debl_res1['ci_len_full'])/np.sqrt(1000)]
            
            ## Ridge projection
            if (i == 0) or (i == 2):
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, Observed)', 
                                                   np.mean(rproj_res1['ci_len_obs1']), 
                                                   np.std(rproj_res1['ci_len_obs1']), 
                                                   np.std(rproj_res1['ci_len_obs1'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(rproj_res1['ci_len_ipw1']), 
                                                   np.std(rproj_res1['ci_len_ipw1']), 
                                                   np.std(rproj_res1['ci_len_ipw1'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, Observed)', np.mean(rproj_res1['ci_len_obs2']), 
                                                   np.std(rproj_res1['ci_len_obs2']), 
                                                   np.std(rproj_res1['ci_len_obs2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, IPW)', np.mean(rproj_res1['ci_len_ipw2']), 
                                                   np.std(rproj_res1['ci_len_ipw2']), 
                                                   np.std(rproj_res1['ci_len_ipw2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (Full oracle data)', np.mean(rproj_res1['ci_len_full']), 
                                                   np.std(rproj_res1['ci_len_full']), 
                                                   np.std(rproj_res1['ci_len_full'])/np.sqrt(1000)]
            
            ## Lasso refitting
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(refit_res1['ci_len_obs1']), 
                                               np.std(refit_res1['ci_len_obs1']), 
                                               np.std(refit_res1['ci_len_obs1'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(refit_res1['ci_len_ipw1']), 
                                               np.std(refit_res1['ci_len_ipw1']), 
                                               np.std(refit_res1['ci_len_ipw1'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(refit_res1['ci_len_obs2']), 
                                               np.std(refit_res1['ci_len_obs2']), 
                                               np.std(refit_res1['ci_len_obs2'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(refit_res1['ci_len_ipw2']), 
                                               np.std(refit_res1['ci_len_ipw2']), 
                                               np.std(refit_res1['ci_len_ipw2'])/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (Full oracle data)', np.mean(refit_res1['ci_len_full']), 
                                               np.std(refit_res1['ci_len_full']), 
                                               np.std(refit_res1['ci_len_full'])/np.sqrt(1000)]
            
            ## Proposed debiasing framework
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, Mincv)', 
                                               np.mean(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, Mincv)', 
                                               np.mean(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, 1SE)', 
                                               np.mean(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, 1SE)', 
                                               np.mean(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, Min-feas)', 
                                               np.mean(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, Min-feas)', 
                                               np.mean(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                               np.std(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
            
            full_res1 = pd.concat([Bias1, Cov_prob1[['Coverage Probability', 'Std', 'StdErr']], 
                                   CI_len1[['CI length', 'Std', 'StdErr']]], axis=1)
            
            # full_res1.to_csv('./Results/Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv', index=False)
            full_res1.to_excel(writer, sheet_name='Cirsym_cov_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k), index=False)

In [6]:
np.sqrt(1000)

31.622776601683793

### d=1000, n=900 (other types of noises)

In [4]:
## Homoscedastic case (t-distributed noises)
d = 1000
n = 900
noises_type = ['laperr', 'uniferr', 'terr']
for noise in noises_type:
    with pd.ExcelWriter('./Results/Cirsym_Full_Results_d'+str(d)+'_n'+str(n)+'_'+str(noise)+'.xlsx') as writer:
        for i in range(5):
            if i == 0:
                ## x0
                x = np.zeros((d,))
                x[0] = 1
            if i == 1:
                ## x1
                x = np.zeros((d,))
                x[0] = 1
                x[1] = 1/2
                x[2] = 1/4
                x[6] = 1/2
                x[7] = 1/8
            if i == 2:
                ## x2
                x = np.zeros((d,))
                x[99] = 1
            if i == 3:
                ## x3
                x = 1/np.linspace(1, d, d)
            if i == 4:
                ## x4
                x = 1/np.linspace(1, d, d)**2
            for k in range(3):
                if k == 0:
                    s_beta = 5
                    beta_0 = np.zeros((d,))
                    beta_0[:s_beta] = np.sqrt(5)
                if k == 1:
                    beta_0 = 1/np.sqrt(np.linspace(1, d, d))
                    beta_0 = 5*beta_0/np.linalg.norm(beta_0)
                if k == 2:
                    beta_0 = 1/np.linspace(1, d, d)
                    beta_0 = 5*beta_0/np.linalg.norm(beta_0)
        
                # True regression function
                m_true = np.dot(x, beta_0)
                
                debl_res1 = pd.read_csv('./Results/debl_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'.csv')
                refit_res1 = pd.read_csv('./Results/refit_Cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'.csv')
                deb_est_1se = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'_1se.csv')
                deb_est_mincv = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'_mincv.csv')
                deb_est_minfeas = pd.read_csv('./Results/DebiasProg_Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'_minfeas.csv')
                if (i == 0) or (i == 2):
                    rproj_res1 = pd.read_csv('./Results/rproj_cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'_'+str(noise)+'.csv')
                    
                # Bias Comparisons
                ## Debiased lasso (Javanmard and Montarani, 2014)
                debl_obs1 = np.mean(abs(debl_res1['m_obs1'] - m_true))
                debl_obs_std1 = np.std(abs(debl_res1['m_obs1'] - m_true))
                debl_obs_ste1 = debl_obs_std1/np.sqrt(1000)
                # Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', debl_obs1, debl_obs_std1, debl_obs_ste1]
                Bias1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                      'Avg bias': [debl_obs1], 'Std': [debl_obs_std1], 'StdErr': [debl_obs_ste1]})
                
                debl_ipw1 = np.mean(abs(debl_res1['m_ipw1'] - m_true))
                debl_ipw_std1 = np.std(abs(debl_res1['m_ipw1'] - m_true))
                debl_ipw_ste1 = debl_ipw_std1/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', debl_ipw1, 
                                               debl_ipw_std1, debl_ipw_ste1]
                
                debl_obs2 = np.mean(abs(debl_res1['m_obs2'] - m_true))
                debl_obs_std2 = np.std(abs(debl_res1['m_obs2'] - m_true))
                debl_obs_ste2 = debl_obs_std2 / np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', debl_obs2, 
                                               debl_obs_std2, debl_obs_ste2]
                
                debl_ipw2 = np.mean(abs(debl_res1['m_ipw2'] - m_true))
                debl_ipw_std2 = np.std(abs(debl_res1['m_ipw2'] - m_true))
                debl_ipw_ste2 = debl_ipw_std2/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', debl_ipw2, 
                                               debl_ipw_std2, debl_ipw_ste2]
                
                debl_full = np.mean(abs(debl_res1['m_full'] - m_true))
                debl_full_std = np.std(abs(debl_res1['m_full'] - m_true))
                debl_full_ste = debl_full_std/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', debl_full, 
                                               debl_full_std, debl_full_ste]
                
                ## Ridge projection
                if (i == 0) or (i == 2):
                    rproj_obs1 = np.mean(abs(rproj_res1['m_obs1'] - m_true))
                    rproj_obs_std1 = np.std(abs(rproj_res1['m_obs1'] - m_true))
                    rproj_obs_ste1 = rproj_obs_std1/np.sqrt(1000)
                    Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, Observed)', rproj_obs1, rproj_obs_std1, 
                                                   rproj_obs_ste1]
                    
                    rproj_ipw1 = np.mean(abs(rproj_res1['m_ipw1'] - m_true))
                    rproj_ipw_std1 = np.std(abs(rproj_res1['m_ipw1'] - m_true))
                    rproj_ipw_ste1 = rproj_ipw_std1/np.sqrt(1000)
                    Bias1.loc[len(Bias1.index)] = ['Ridge projection (MCAR, IPW)', rproj_ipw1, rproj_ipw_std1, rproj_ipw_ste1]
                    
                    rproj_obs2 = np.mean(abs(rproj_res1['m_obs2'] - m_true))
                    rproj_obs_std2 = np.std(abs(rproj_res1['m_obs2'] - m_true))
                    rproj_obs_ste2 = rproj_obs_std2/np.sqrt(1000)
                    Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, Observed)', rproj_obs2, rproj_obs_std2, 
                                                   rproj_obs_ste2]
                    
                    rproj_ipw2 = np.mean(abs(rproj_res1['m_ipw2'] - m_true))
                    rproj_ipw_std2 = np.std(abs(rproj_res1['m_ipw2'] - m_true))
                    rproj_ipw_ste2 = rproj_ipw_std2/np.sqrt(1000)
                    Bias1.loc[len(Bias1.index)] = ['Ridge projection (MAR, IPW)', rproj_ipw2, rproj_ipw_std2, rproj_ipw_ste2]
                    
                    rproj_full = np.mean(abs(rproj_res1['m_full'] - m_true))
                    rproj_full_std = np.std(abs(rproj_res1['m_full'] - m_true))
                    rproj_full_ste = rproj_full_std / np.sqrt(1000)
                    Bias1.loc[len(Bias1.index)] = ['Ridge projection (Full oracle data)', rproj_full, 
                                                   rproj_full_std, rproj_full_ste]
                
                ## Lasso Refitting
                refit_obs1 = np.mean(abs(refit_res1['m_obs1'] - m_true))
                refit_obs_std1 = np.std(abs(refit_res1['m_obs1'] - m_true))
                refit_obs_ste1 = refit_obs_std1/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, Observed)', refit_obs1, refit_obs_std1, refit_obs_ste1]
                
                refit_ipw1 = np.mean(abs(refit_res1['m_ipw1'] - m_true))
                refit_ipw_std1 = np.std(abs(refit_res1['m_ipw1'] - m_true))
                refit_ipw_ste1 = refit_ipw_std1/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, IPW)', refit_ipw1, refit_ipw_std1, refit_ipw_ste1]
                
                refit_obs2 = np.mean(abs(refit_res1['m_obs2'] - m_true))
                refit_obs_std2 = np.std(abs(refit_res1['m_obs2'] - m_true))
                refit_obs_ste2 = refit_obs_std2/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, Observed)', refit_obs2, refit_obs_std2, refit_obs_ste2]
                
                refit_ipw2 = np.mean(abs(refit_res1['m_ipw2'] - m_true))
                refit_ipw_std2 = np.std(abs(refit_res1['m_ipw2'] - m_true))
                refit_ipw_ste2 = refit_ipw_std2/np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, IPW)', refit_ipw2, refit_ipw_std2, refit_obs_ste2]
                
                refit_full = np.mean(abs(refit_res1['m_full'] - m_true))
                refit_full_std = np.std(abs(refit_res1['m_full'] - m_true))
                refit_full_ste = refit_full_std / np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (Full oracle data)', refit_full, refit_full_std, refit_full_ste]
                
                ## Our proposed debiasing framework
                m1 = np.mean(abs(deb_est_mincv['m_deb1'] - m_true))
                m_std1 = np.std(abs(deb_est_mincv['m_deb1'] - m_true))
                m_ste1 = m_std1 /np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, Mincv)', m1, m_std1, m_ste1]
                
                m2 = np.mean(abs(deb_est_mincv['m_deb2'] - m_true))
                m_std2 = np.std(abs(deb_est_mincv['m_deb2'] - m_true))
                m_ste2 = m_std2 / np.sqrt(1000) 
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, Mincv)', m2, m_std2, m_ste2]
                
                m1 = np.mean(abs(deb_est_1se['m_deb1'] - m_true))
                m_std1 = np.std(abs(deb_est_1se['m_deb1'] - m_true))
                m_ste1 = m_std1 /np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, 1SE)', m1, m_std1, m_ste1]
                
                m2 = np.mean(abs(deb_est_1se['m_deb2'] - m_true))
                m_std2 = np.std(abs(deb_est_1se['m_deb2'] - m_true))
                m_ste2 = m_std2 / np.sqrt(1000) 
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, 1SE)', m2, m_std2, m_ste2]
                
                m1 = np.mean(abs(deb_est_minfeas['m_deb1'] - m_true))
                m_std1 = np.std(abs(deb_est_minfeas['m_deb1'] - m_true))
                m_ste1 = m_std1 /np.sqrt(1000)
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MCAR, Min-feas)', m1, m_std1, m_ste1]
                
                m2 = np.mean(abs(deb_est_minfeas['m_deb2'] - m_true))
                m_std2 = np.std(abs(deb_est_minfeas['m_deb2'] - m_true))
                m_ste2 = m_std2 / np.sqrt(1000) 
                Bias1.loc[len(Bias1.index)] = ['Proposed framework (MAR, Min-feas)', m2, m_std2, m_ste2]
            
            
                # Coverage Probability
                ## Debiased lasso (Javanmard and Montarani, 2014)
                cov_prob = (debl_res1['m_obs1'] + debl_res1['ci_len_obs1']/2 >= m_true) & \
                (debl_res1['m_obs1'] - debl_res1['ci_len_obs1']/2 <= m_true)
                # Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
                Cov_prob1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                          'Coverage Probability': [np.mean(cov_prob)], 
                                          'Std': [np.std(cov_prob)], 
                                          'StdErr': [np.std(cov_prob)/np.sqrt(1000)]})
                
                cov_prob = (debl_res1['m_ipw1'] + debl_res1['ci_len_ipw1']/2 >= m_true) & \
                (debl_res1['m_ipw1'] - debl_res1['ci_len_ipw1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (debl_res1['m_obs2'] + debl_res1['ci_len_obs2']/2 >= m_true) & \
                (debl_res1['m_obs2'] - debl_res1['ci_len_obs2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (debl_res1['m_ipw2'] + debl_res1['ci_len_ipw2']/2 >= m_true) & \
                (debl_res1['m_ipw2'] - debl_res1['ci_len_ipw2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (debl_res1['m_full'] + debl_res1['ci_len_full']/2 >= m_true) & \
                (debl_res1['m_full'] - debl_res1['ci_len_full']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Debiased lasso (Javanmard) (Full data)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                ## Ridge projection
                if (i == 0) or (i == 2):
                    cov_prob = (rproj_res1['m_obs1'] + rproj_res1['ci_len_obs1']/2 >= m_true) & \
                    (rproj_res1['m_obs1'] - rproj_res1['ci_len_obs1']/2 <= m_true)
                    Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, Observed)', np.mean(cov_prob), 
                                                           np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                    
                    cov_prob = (rproj_res1['m_ipw1'] + rproj_res1['ci_len_ipw1']/2 >= m_true) & \
                    (rproj_res1['m_ipw1'] - rproj_res1['ci_len_ipw1']/2 <= m_true)
                    Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(cov_prob), 
                                                           np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                    
                    cov_prob = (rproj_res1['m_obs2'] + rproj_res1['ci_len_obs2']/2 >= m_true) & \
                    (rproj_res1['m_obs2'] - rproj_res1['ci_len_obs2']/2 <= m_true)
                    Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, Observed)', np.mean(cov_prob), 
                                                           np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                    
                    cov_prob = (rproj_res1['m_ipw2'] + rproj_res1['ci_len_ipw2']/2 >= m_true) & \
                    (rproj_res1['m_ipw2'] - rproj_res1['ci_len_ipw2']/2 <= m_true)
                    Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (MAR, IPW)', np.mean(cov_prob), 
                                                           np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                    
                    cov_prob = (rproj_res1['m_full'] + rproj_res1['ci_len_full']/2 >= m_true) & \
                    (rproj_res1['m_full'] - rproj_res1['ci_len_full']/2 <= m_true)
                    Cov_prob1.loc[len(Cov_prob1.index)] = ['Ridge projection (Full data)', np.mean(cov_prob), 
                                                           np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                ## Lasso refitting
                cov_prob = (refit_res1['m_obs1'] + refit_res1['ci_len_obs1']/2 >= m_true) & \
                (refit_res1['m_obs1'] - refit_res1['ci_len_obs1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (refit_res1['m_ipw1'] + refit_res1['ci_len_ipw1']/2 >= m_true) & \
                (refit_res1['m_ipw1'] - refit_res1['ci_len_ipw1']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (refit_res1['m_obs2'] + refit_res1['ci_len_obs2']/2 >= m_true) & \
                (refit_res1['m_obs2'] - refit_res1['ci_len_obs2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (refit_res1['m_ipw2'] + refit_res1['ci_len_ipw2']/2 >= m_true) & \
                (refit_res1['m_ipw2'] - refit_res1['ci_len_ipw2']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (refit_res1['m_full'] + refit_res1['ci_len_full']/2 >= m_true) & \
                (refit_res1['m_full'] - refit_res1['ci_len_full']/2 <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (Full data)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                # Proposed framework   
                cov_prob = (deb_est_mincv['m_deb1'] + deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_mincv['m_deb1'] - deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, Mincv)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (deb_est_mincv['m_deb2'] + deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_mincv['m_deb2'] - deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, Mincv)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (deb_est_1se['m_deb1'] + deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_1se['m_deb1'] - deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, 1SE)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (deb_est_1se['m_deb2'] + deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_1se['m_deb2'] - deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, 1SE)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (deb_est_minfeas['m_deb1'] + deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_minfeas['m_deb1'] - deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MCAR, Min-feas)', np.mean(cov_prob), 
                                                       np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
                
                cov_prob = (deb_est_minfeas['m_deb2'] + deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) >= m_true) & \
                (deb_est_minfeas['m_deb2'] - deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2) <= m_true)
                Cov_prob1.loc[len(Cov_prob1.index)] = ['Proposed framework (MAR, Min-feas)', np.mean(cov_prob), 
                                                   np.std(cov_prob), np.std(cov_prob)/np.sqrt(1000)]
            
        
            
                # Lengths of confidence intervals
                ## Debiased lasso (Javanmard and Montarani, 2014)
                CI_len1 = pd.DataFrame({'Scenario': ['Debiased lasso (Javanmard) (MCAR, Observed)'], 
                                'CI length': [np.mean(debl_res1['ci_len_obs1'])], 
                                'Std': [np.std(debl_res1['ci_len_obs1'])], 
                                'StdErr': [np.std(debl_res1['ci_len_obs1'])/np.sqrt(1000)]})
    #             CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, Observed)', np.mean(debl_res1['ci_len_obs1']), 
    #                                                np.std(debl_res1['ci_len_obs1'])]
                CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MCAR, IPW)', np.mean(debl_res1['ci_len_ipw1']), 
                                                   np.std(debl_res1['ci_len_ipw1']), 
                                                   np.std(debl_res1['ci_len_ipw1'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, Observed)', np.mean(debl_res1['ci_len_obs2']), 
                                                   np.std(debl_res1['ci_len_obs2']), 
                                                   np.std(debl_res1['ci_len_obs2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (MAR, IPW)', np.mean(debl_res1['ci_len_ipw2']), 
                                                   np.std(debl_res1['ci_len_ipw2']), 
                                                   np.std(debl_res1['ci_len_ipw2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Debiased lasso (Javanmard) (Full oracle data)', np.mean(debl_res1['ci_len_full']), 
                                                   np.std(debl_res1['ci_len_full']), np.std(debl_res1['ci_len_full'])/np.sqrt(1000)]
                
                ## Ridge projection
                if (i == 0) or (i == 2):
                    CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, Observed)', 
                                                       np.mean(rproj_res1['ci_len_obs1']), 
                                                       np.std(rproj_res1['ci_len_obs1']), 
                                                       np.std(rproj_res1['ci_len_obs1'])/np.sqrt(1000)]
                    CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MCAR, IPW)', np.mean(rproj_res1['ci_len_ipw1']), 
                                                       np.std(rproj_res1['ci_len_ipw1']), 
                                                       np.std(rproj_res1['ci_len_ipw1'])/np.sqrt(1000)]
                    CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, Observed)', np.mean(rproj_res1['ci_len_obs2']), 
                                                       np.std(rproj_res1['ci_len_obs2']), 
                                                       np.std(rproj_res1['ci_len_obs2'])/np.sqrt(1000)]
                    CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (MAR, IPW)', np.mean(rproj_res1['ci_len_ipw2']), 
                                                       np.std(rproj_res1['ci_len_ipw2']), 
                                                       np.std(rproj_res1['ci_len_ipw2'])/np.sqrt(1000)]
                    CI_len1.loc[len(CI_len1.index)] = ['Ridge projection (Full oracle data)', np.mean(rproj_res1['ci_len_full']), 
                                                       np.std(rproj_res1['ci_len_full']), 
                                                       np.std(rproj_res1['ci_len_full'])/np.sqrt(1000)]
                
                ## Lasso refitting
                CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, Observed)', np.mean(refit_res1['ci_len_obs1']), 
                                                   np.std(refit_res1['ci_len_obs1']), 
                                                   np.std(refit_res1['ci_len_obs1'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(refit_res1['ci_len_ipw1']), 
                                                   np.std(refit_res1['ci_len_ipw1']), 
                                                   np.std(refit_res1['ci_len_ipw1'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(refit_res1['ci_len_obs2']), 
                                                   np.std(refit_res1['ci_len_obs2']), 
                                                   np.std(refit_res1['ci_len_obs2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(refit_res1['ci_len_ipw2']), 
                                                   np.std(refit_res1['ci_len_ipw2']), 
                                                   np.std(refit_res1['ci_len_ipw2'])/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (Full oracle data)', np.mean(refit_res1['ci_len_full']), 
                                                   np.std(refit_res1['ci_len_full']), 
                                                   np.std(refit_res1['ci_len_full'])/np.sqrt(1000)]
                
                ## Proposed debiasing framework
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, Mincv)', 
                                                   np.mean(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_mincv['asym_se1']*deb_est_mincv['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, Mincv)', 
                                                   np.mean(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_mincv['asym_se2']*deb_est_mincv['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, 1SE)', 
                                                   np.mean(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_1se['asym_se1']*deb_est_1se['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, 1SE)', 
                                                   np.mean(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_1se['asym_se2']*deb_est_1se['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MCAR, Min-feas)', 
                                                   np.mean(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_minfeas['asym_se1']*deb_est_minfeas['sigma_hat1']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                CI_len1.loc[len(CI_len1.index)] = ['Proposed framework (MAR, Min-feas)', 
                                                   np.mean(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2)), 
                                                   np.std(2*deb_est_minfeas['asym_se2']*deb_est_minfeas['sigma_hat2']*scipy.stats.norm.ppf(1-0.05/2))/np.sqrt(1000)]
                
                full_res1 = pd.concat([Bias1, Cov_prob1[['Coverage Probability', 'Std', 'StdErr']], 
                                       CI_len1[['CI length', 'Std', 'StdErr']]], axis=1)
                
                # full_res1.to_csv('./Results/Cirsym_cov_homoerr_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k)+'.csv', index=False)
                full_res1.to_excel(writer, sheet_name='Cirsym_cov_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta'+str(k), index=False)

### Lasso Refitting (Increasing the noise levels)

In [2]:
## Homoscedastic case
d = 1000
n = 900
with pd.ExcelWriter('./Results/Lasso_Refit_Cirsym_Full_Results_d'+str(d)+'_n'+str(n)+'.xlsx') as writer:
    for i in range(4):
        if i == 0:
            ## x0
            x = np.zeros((d,))
            x[0] = 1
        if i == 1:
            ## x1
            x = np.zeros((d,))
            x[0] = 1
            x[1] = 1/2
            x[2] = 1/4
            x[6] = 1/2
            x[7] = 1/8
        if i == 2:
            ## x2
            x = np.zeros((d,))
            x[99] = 1
        if i == 3:
            ## x3
            x = 1/np.linspace(1, d, d)**2
            
        k = 0
        s_beta = 5
        beta_0 = np.zeros((d,))
        beta_0[:s_beta] = np.sqrt(5)
        
        for sig in [1,2,3,4,5]:
            # True regression function
            m_true = np.dot(x, beta_0)
            refit_res1 = pd.read_csv('./Results/refit_Cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta0_sig_'+str(sig)+'.csv')
            
            ## Lasso Refitting
            refit_obs1 = np.mean(abs(refit_res1['m_obs1'] - m_true))
            refit_obs_std1 = np.std(abs(refit_res1['m_obs1'] - m_true))
            Bias1 = pd.DataFrame({'Scenario': ['Lasso Refitting (MCAR, Observed)'], 
                                  'Avg bias': [refit_obs1], 'Std': [refit_obs_std1]})
            
            refit_ipw1 = np.mean(abs(refit_res1['m_ipw1'] - m_true))
            refit_ipw_std1 = np.std(abs(refit_res1['m_ipw1'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MCAR, IPW)', refit_ipw1, refit_ipw_std1]
            
            refit_obs2 = np.mean(abs(refit_res1['m_obs2'] - m_true))
            refit_obs_std2 = np.std(abs(refit_res1['m_obs2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, Observed)', refit_obs2, refit_obs_std2]
            
            refit_ipw2 = np.mean(abs(refit_res1['m_ipw2'] - m_true))
            refit_ipw_std2 = np.std(abs(refit_res1['m_ipw2'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (MAR, IPW)', refit_ipw2, refit_ipw_std2]
            
            refit_full = np.mean(abs(refit_res1['m_full'] - m_true))
            refit_full_std = np.std(abs(refit_res1['m_full'] - m_true))
            Bias1.loc[len(Bias1.index)] = ['Lasso Refitting (Full oracle data)', refit_full, refit_full_std]
            
            
            ## Lasso refitting
            cov_prob = (refit_res1['m_obs1'] + refit_res1['ci_len_obs1']/2 >= m_true) & \
            (refit_res1['m_obs1'] - refit_res1['ci_len_obs1']/2 <= m_true)
            Cov_prob1 = pd.DataFrame({'Scenario': ['Lasso refitting (MCAR, Observed)'], 
                                      'Coverage Probability': [np.mean(cov_prob)], 
                                      'Std': [np.std(cov_prob)]})
            
            cov_prob = (refit_res1['m_ipw1'] + refit_res1['ci_len_ipw1']/2 >= m_true) & \
            (refit_res1['m_ipw1'] - refit_res1['ci_len_ipw1']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_obs2'] + refit_res1['ci_len_obs2']/2 >= m_true) & \
            (refit_res1['m_obs2'] - refit_res1['ci_len_obs2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_ipw2'] + refit_res1['ci_len_ipw2']/2 >= m_true) & \
            (refit_res1['m_ipw2'] - refit_res1['ci_len_ipw2']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(cov_prob), np.std(cov_prob)]
            
            cov_prob = (refit_res1['m_full'] + refit_res1['ci_len_full']/2 >= m_true) & \
            (refit_res1['m_full'] - refit_res1['ci_len_full']/2 <= m_true)
            Cov_prob1.loc[len(Cov_prob1.index)] = ['Lasso refitting (Full data)', np.mean(cov_prob), np.std(cov_prob)]
            
            
            ## Lasso refitting
            CI_len1 = pd.DataFrame({'Scenario': ['Lasso refitting (MCAR, Observed)'], 
                            'CI length': [np.mean(refit_res1['ci_len_obs1'])], 
                            'Std': [np.std(refit_res1['ci_len_obs1'])]})
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MCAR, IPW)', np.mean(refit_res1['ci_len_ipw1']), 
                                               np.std(refit_res1['ci_len_ipw1'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, Observed)', np.mean(refit_res1['ci_len_obs2']), 
                                               np.std(refit_res1['ci_len_obs2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (MAR, IPW)', np.mean(refit_res1['ci_len_ipw2']), 
                                               np.std(refit_res1['ci_len_ipw2'])]
            CI_len1.loc[len(CI_len1.index)] = ['Lasso refitting (Full oracle data)', np.mean(refit_res1['ci_len_full']), 
                                               np.std(refit_res1['ci_len_full'])]
            
            full_res1 = pd.concat([Bias1, Cov_prob1[['Coverage Probability', 'Std']], CI_len1[['CI length', 'Std']]], axis=1)
            
            full_res1.to_excel(writer, sheet_name='Cirsym_d'+str(d)+'_n'+str(n)+'_x'+str(i)+'_beta0_sig_'+str(sig), index=False)
            

/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.w